In [1]:
import pandas as pd

# Mini reto 001

### Ordenar las subseries de forma descendente

La finalidad es ordenar de forma descendente las subseries 1..n que hay en la serie.

Output solicitado:

[6,5,4,3,2,1,4,3,2,1,5,4,3,2,1,2,1,9,8,7,6,5,4,3,2,1]

In [2]:
serie = pd.Series([1,2,3,4,5,6,1,2,3,4,1,2,3,4,5,1,2,1,2,3,4,5,6,7,8,9])
serie

0     1
1     2
2     3
3     4
4     5
5     6
6     1
7     2
8     3
9     4
10    1
11    2
12    3
13    4
14    5
15    1
16    2
17    1
18    2
19    3
20    4
21    5
22    6
23    7
24    8
25    9
dtype: int64

## Solucion Propuesta por Paduel

Solución en una linea.

In [3]:
serie.groupby(serie.diff().shift(-1).fillna(-1).lt(0).replace(
    False, pd.NA).mul(serie.index).bfill()).apply(
        lambda x:x[::-1]).set_axis(serie.index)

0     6
1     5
2     4
3     3
4     2
5     1
6     4
7     3
8     2
9     1
10    5
11    4
12    3
13    2
14    1
15    2
16    1
17    9
18    8
19    7
20    6
21    5
22    4
23    3
24    2
25    1
dtype: int64

Lo mismo pero más legible. Mejor usar esta forma en nuestro código es mas limpio.

In [4]:
maxis = serie.diff().shift(-1).fillna(-1).lt(0)
groups = maxis.replace(False, pd.NA).mul(serie.index).bfill()
reordenada = serie.groupby(groups).apply(lambda x:x[::-1]).set_axis(serie.index)

In [5]:
reordenada

0     6
1     5
2     4
3     3
4     2
5     1
6     4
7     3
8     2
9     1
10    5
11    4
12    3
13    2
14    1
15    2
16    1
17    9
18    8
19    7
20    6
21    5
22    4
23    3
24    2
25    1
dtype: int64

## Paso a paso

Detectamos los sucesivos máximos en `serie`, simplemente comprobando su diferencia con el siguiente numero. Asignamos -1 al último valor, pues se produce un np.nan. Y comprobamos cuales son menores que cero.

In [6]:
maxis = serie.diff().shift(-1).fillna(-1).lt(0)

maxis

0     False
1     False
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14     True
15    False
16     True
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25     True
dtype: bool

Para identificar los componentes de los grupos reemplazamos los False por pd.NA, multiplicamos los True por su indice y finalmente rellenamos los p.NA hacia atras.

In [7]:
groups = maxis.replace(False, pd.NA).mul(serie.index).bfill()

groups

0      5.0
1      5.0
2      5.0
3      5.0
4      5.0
5      5.0
6      9.0
7      9.0
8      9.0
9      9.0
10    14.0
11    14.0
12    14.0
13    14.0
14    14.0
15    16.0
16    16.0
17    25.0
18    25.0
19    25.0
20    25.0
21    25.0
22    25.0
23    25.0
24    25.0
25    25.0
dtype: float64

Finalmente agrupamos `serie` en base a los grupos creados y los invertimos usando `apply` y le asignamos de nuevo el indice inial para dejarlo listo.

In [8]:
reordenada = serie.groupby(groups).apply(lambda x:x[::-1]).set_axis(serie.index)

reordenada

0     6
1     5
2     4
3     3
4     2
5     1
6     4
7     3
8     2
9     1
10    5
11    4
12    3
13    2
14    1
15    2
16    1
17    9
18    8
19    7
20    6
21    5
22    4
23    3
24    2
25    1
dtype: int64

Podríamos usar `.cumcount(ascending=False)` en vez de ese `apply` pero de esta forma sirve para series crecientes incluso cuando el paso de incremento sea mayor que uno. Obteniendo una solución más general.